In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

# Define column names as per UCI dataset description
column_names = [
    "ID", "Diagnosis", "Radius_mean", "Texture_mean", "Perimeter_mean", "Area_mean",
    "Smoothness_mean", "Compactness_mean", "Concavity_mean", "Concave_points_mean",
    "Symmetry_mean", "Fractal_dimension_mean", "Radius_se", "Texture_se", "Perimeter_se",
    "Area_se", "Smoothness_se", "Compactness_se", "Concavity_se", "Concave_points_se",
    "Symmetry_se", "Fractal_dimension_se", "Radius_worst", "Texture_worst", "Perimeter_worst",
    "Area_worst", "Smoothness_worst", "Compactness_worst", "Concavity_worst",
    "Concave_points_worst", "Symmetry_worst", "Fractal_dimension_worst"
]

# Load the dataset
df = pd.read_csv("wdbc.data", header=None, names=column_names)

# Save to CSV with column names
df.to_csv("breast_cancer_diagnostic.csv", index=False)

# Display the first few rows
print(df.head())

         ID Diagnosis  Radius_mean  Texture_mean  Perimeter_mean  Area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   Smoothness_mean  Compactness_mean  Concavity_mean  Concave_points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  Radius_worst  Texture_worst  Perimeter_worst  Area_wor

# mRMR Feature Selection with Grid Search

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mrmr import mrmr_classif
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# ✅ Custom Transformer for mRMR Feature Selection
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, K=10):
        self.K = K
        self.selected_features = None

    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):  # Ensure X is a DataFrame
            raise ValueError("X must be a Pandas DataFrame for mRMR")
        self.selected_features = mrmr_classif(X=X, y=y, K=self.K)
        return self

    def transform(self, X):
        return X[self.selected_features]

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models
models = [
    ('LR', LogisticRegression(max_iter=1000)),
    ('LDA', LinearDiscriminantAnalysis()),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('NB', GaussianNB()),
    ('SVM', SVC())
]

cv_folds = 10  # Use 5-fold cross-validation
best_overall_score = 0
best_K = None
best_model_name = None

# ✅ Iterate over K values from 5 to 27
for K in range(5, 28):
    print(f"\nEvaluating K={K}...\n")
    
    # Feature Selection
    selector = MRMRFeatureSelector(K=K)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Standardize After Feature Selection
    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, model in models:
        # Perform Cross-Validation
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy')
        mean_accuracy = np.mean(cv_scores)

        # Train on full training set
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"K={K}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        # ✅ Track best-performing model
        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_K = K
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with K={best_K}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating K=5...



100%|█████████████████████████████████████████████| 5/5 [00:00<00:00, 68.84it/s]


K=5, LR: CV Accuracy = 0.9407, Test Accuracy = 0.9737
K=5, LDA: CV Accuracy = 0.9428, Test Accuracy = 0.9474
K=5, KNN: CV Accuracy = 0.9451, Test Accuracy = 0.9649
K=5, CART: CV Accuracy = 0.9297, Test Accuracy = 0.9386
K=5, NB: CV Accuracy = 0.9428, Test Accuracy = 0.9474
K=5, SVM: CV Accuracy = 0.9494, Test Accuracy = 0.9561

Evaluating K=6...



100%|█████████████████████████████████████████████| 6/6 [00:00<00:00, 63.54it/s]


K=6, LR: CV Accuracy = 0.9472, Test Accuracy = 0.9737
K=6, LDA: CV Accuracy = 0.9472, Test Accuracy = 0.9737
K=6, KNN: CV Accuracy = 0.9451, Test Accuracy = 0.9649
K=6, CART: CV Accuracy = 0.9122, Test Accuracy = 0.9386
K=6, NB: CV Accuracy = 0.9406, Test Accuracy = 0.9474
K=6, SVM: CV Accuracy = 0.9494, Test Accuracy = 0.9561

Evaluating K=7...



...

K=25, LR: CV Accuracy = 0.9736, Test Accuracy = 0.9737
K=25, LDA: CV Accuracy = 0.9495, Test Accuracy = 0.9561
K=25, KNN: CV Accuracy = 0.9604, Test Accuracy = 0.9649
K=25, CART: CV Accuracy = 0.9363, Test Accuracy = 0.9035
K=25, NB: CV Accuracy = 0.9230, Test Accuracy = 0.9649
K=25, SVM: CV Accuracy = 0.9670, Test Accuracy = 0.9737

Evaluating K=26...



100%|███████████████████████████████████████████| 26/26 [00:00<00:00, 26.38it/s]


K=26, LR: CV Accuracy = 0.9758, Test Accuracy = 0.9737
K=26, LDA: CV Accuracy = 0.9561, Test Accuracy = 0.9561
K=26, KNN: CV Accuracy = 0.9626, Test Accuracy = 0.9649
K=26, CART: CV Accuracy = 0.9298, Test Accuracy = 0.9386
K=26, NB: CV Accuracy = 0.9252, Test Accuracy = 0.9649
K=26, SVM: CV Accuracy = 0.9670, Test Accuracy = 0.9649

Evaluating K=27...



100%|███████████████████████████████████████████| 27/27 [00:01<00:00, 16.66it/s]


K=27, LR: CV Accuracy = 0.9758, Test Accuracy = 0.9737
K=27, LDA: CV Accuracy = 0.9539, Test Accuracy = 0.9561
K=27, KNN: CV Accuracy = 0.9648, Test Accuracy = 0.9649
K=27, CART: CV Accuracy = 0.9342, Test Accuracy = 0.9474
K=27, NB: CV Accuracy = 0.9340, Test Accuracy = 0.9649
K=27, SVM: CV Accuracy = 0.9670, Test Accuracy = 0.9649

🎯 Best Model: SVM with K=18
📈 Best Cross-Validation Accuracy: 0.9802


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mrmr import mrmr_classif
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

# ✅ Custom Transformer for mRMR Feature Selection
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, K=10):
        self.K = K
        self.selected_features = None

    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):  # Ensure X is a DataFrame
            raise ValueError("X must be a Pandas DataFrame for mRMR")
        self.selected_features = mrmr_classif(X=X, y=y, K=self.K)
        return self

    def transform(self, X):
        return X[self.selected_features]

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Search Space for K (Randomized Search)
param_dist = {'K': randint(5, 28)}  # Randomly choose K from 5 to 27

# ✅ Perform Randomized Search for Best K
random_search = RandomizedSearchCV(
    MRMRFeatureSelector(), param_distributions=param_dist, 
    n_iter=10, cv=10, scoring='accuracy', 
    random_state=42, n_jobs=-1, verbose=1
)
random_search.fit(X_train, y_train)

# ✅ Get the Best K
best_K = random_search.best_params_['K']
print(f"Best Number of Features (K): {best_K}")

# ✅ Use best K to select features
selector = MRMRFeatureSelector(K=best_K)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# ✅ Standardize After Feature Selection
sc = StandardScaler()
X_train_selected = sc.fit_transform(X_train_selected)
X_test_selected = sc.transform(X_test_selected)

# ✅ Define Models
models = [
    ('LR', LogisticRegression(max_iter=1000)),
    ('LDA', LinearDiscriminantAnalysis()),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('NB', GaussianNB()),
    ('SVM', SVC())
]

cv_folds = 10  # Use 10-fold cross-validation
best_overall_score = 0
best_model_name = None

# ✅ Train and Evaluate Models
for name, model in models:
    # Perform Cross-Validation
    cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy')
    mean_accuracy = np.mean(cv_scores)

    # Train on full training set
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"{name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

    # ✅ Track best-performing model
    if mean_accuracy > best_overall_score:
        best_overall_score = mean_accuracy
        best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with K={best_K}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")


Fitting 10 folds for each of 10 candidates, totalling 100 fits


100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 43.80it/s]


Best Number of Features (K): 11


100%|███████████████████████████████████████████| 11/11 [00:00<00:00, 62.42it/s]


LR: CV Accuracy = 0.9516, Test Accuracy = 0.9737
LDA: CV Accuracy = 0.9363, Test Accuracy = 0.9561
KNN: CV Accuracy = 0.9362, Test Accuracy = 0.9737
CART: CV Accuracy = 0.9186, Test Accuracy = 0.9298
NB: CV Accuracy = 0.9296, Test Accuracy = 0.9649
SVM: CV Accuracy = 0.9494, Test Accuracy = 0.9561

🎯 Best Model: LR with K=11
📈 Best Cross-Validation Accuracy: 0.9516


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from mrmr import mrmr_classif
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

# ✅ Custom Transformer for mRMR Feature Selection
class MRMRFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, K=10):
        self.K = K
        self.selected_features = None

    def fit(self, X, y):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("X must be a Pandas DataFrame for mRMR")
        self.selected_features = mrmr_classif(X=X, y=y, K=self.K)
        return self

    def transform(self, X):
        return X[self.selected_features]

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models and Hyperparameters for Random Search
models = {
    'LR': (LogisticRegression(max_iter=1000), {'C': np.logspace(-4, 4, 10)}),
    'LDA': (LinearDiscriminantAnalysis(), {}),
    'KNN': (KNeighborsClassifier(), {'n_neighbors': randint(1, 30)}),
    'CART': (DecisionTreeClassifier(), {'max_depth': randint(1, 20)}),
    'NB': (GaussianNB(), {}),
    'SVM': (SVC(), {'C': np.logspace(-4, 4, 10), 'kernel': ['linear', 'rbf']})
}

cv_folds = 10
n_iter_search = 10  # Number of random samples
best_overall_score = 0
best_K = None
best_model_name = None

# ✅ Random Search over K values
for K in np.random.randint(5, 28, 5):  # Randomly selecting 5 values of K
    print(f"\nEvaluating K={K}...\n")
    
    selector = MRMRFeatureSelector(K=K)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, (model, param_dist) in models.items():
        if param_dist:
            random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=n_iter_search, cv=cv_folds, scoring='accuracy', random_state=42)
            random_search.fit(X_train_selected, y_train)
            best_model = random_search.best_estimator_
            mean_accuracy = random_search.best_score_
        else:
            mean_accuracy = np.mean(cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy'))
            model.fit(X_train_selected, y_train)
            best_model = model

        y_pred = best_model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"K={K}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_K = K
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with K={best_K}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating K=16...



100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 62.12it/s]


K=16, LR: CV Accuracy = 0.9758, Test Accuracy = 0.9737
K=16, LDA: CV Accuracy = 0.9496, Test Accuracy = 0.9561
K=16, KNN: CV Accuracy = 0.9670, Test Accuracy = 0.9649
K=16, CART: CV Accuracy = 0.9473, Test Accuracy = 0.9474
K=16, NB: CV Accuracy = 0.9253, Test Accuracy = 0.9649
K=16, SVM: CV Accuracy = 0.9693, Test Accuracy = 0.9561

Evaluating K=16...



100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 63.59it/s]


K=16, LR: CV Accuracy = 0.9758, Test Accuracy = 0.9737
K=16, LDA: CV Accuracy = 0.9496, Test Accuracy = 0.9561
K=16, KNN: CV Accuracy = 0.9670, Test Accuracy = 0.9649
K=16, CART: CV Accuracy = 0.9473, Test Accuracy = 0.9298
K=16, NB: CV Accuracy = 0.9253, Test Accuracy = 0.9649
K=16, SVM: CV Accuracy = 0.9693, Test Accuracy = 0.9561

Evaluating K=5...



...

K=5, LR: CV Accuracy = 0.9582, Test Accuracy = 0.9737
K=5, LDA: CV Accuracy = 0.9428, Test Accuracy = 0.9474
K=5, KNN: CV Accuracy = 0.9516, Test Accuracy = 0.9649
K=5, CART: CV Accuracy = 0.9319, Test Accuracy = 0.9386
K=5, NB: CV Accuracy = 0.9428, Test Accuracy = 0.9474
K=5, SVM: CV Accuracy = 0.9538, Test Accuracy = 0.9825

Evaluating K=13...



100%|███████████████████████████████████████████| 13/13 [00:00<00:00, 69.56it/s]


K=13, LR: CV Accuracy = 0.9757, Test Accuracy = 0.9737
K=13, LDA: CV Accuracy = 0.9496, Test Accuracy = 0.9474
K=13, KNN: CV Accuracy = 0.9626, Test Accuracy = 0.9649
K=13, CART: CV Accuracy = 0.9408, Test Accuracy = 0.9386
K=13, NB: CV Accuracy = 0.9340, Test Accuracy = 0.9649
K=13, SVM: CV Accuracy = 0.9757, Test Accuracy = 0.9737

Evaluating K=16...



100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 66.45it/s]


K=16, LR: CV Accuracy = 0.9758, Test Accuracy = 0.9737
K=16, LDA: CV Accuracy = 0.9496, Test Accuracy = 0.9561
K=16, KNN: CV Accuracy = 0.9670, Test Accuracy = 0.9649
K=16, CART: CV Accuracy = 0.9451, Test Accuracy = 0.9386
K=16, NB: CV Accuracy = 0.9253, Test Accuracy = 0.9649
K=16, SVM: CV Accuracy = 0.9693, Test Accuracy = 0.9561

🎯 Best Model: LR with K=16
📈 Best Cross-Validation Accuracy: 0.9758


# PCA Feature Selection ( Grid Search CV)

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models
models = [
    ('LR', LogisticRegression(max_iter=1000)),
    ('LDA', LinearDiscriminantAnalysis()),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('NB', GaussianNB()),
    ('SVM', SVC())
]

cv_folds = 10  # Use 10-fold cross-validation
best_overall_score = 0
best_n_components = None
best_model_name = None

# ✅ Iterate over the number of principal components (n_components) to test
for n_components in range(5, 28):
    print(f"\nEvaluating n_components={n_components}...\n")
    
    # Feature Selection using PCA
    pca = PCA(n_components=n_components)
    X_train_selected = pca.fit_transform(X_train)
    X_test_selected = pca.transform(X_test)

    # Standardize After Feature Selection (StandardScaler is usually done after PCA)
    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, model in models:
        # Perform Cross-Validation
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy')
        mean_accuracy = np.mean(cv_scores)

        # Train on full training set
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"n_components={n_components}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        # ✅ Track best-performing model
        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_n_components = n_components
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with n_components={best_n_components}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating n_components=5...

n_components=5, LR: CV Accuracy = 0.9428, Test Accuracy = 0.9561
n_components=5, LDA: CV Accuracy = 0.9427, Test Accuracy = 0.9474
n_components=5, KNN: CV Accuracy = 0.9208, Test Accuracy = 0.9561
n_components=5, CART: CV Accuracy = 0.9272, Test Accuracy = 0.9474
n_components=5, NB: CV Accuracy = 0.9054, Test Accuracy = 0.9386
n_components=5, SVM: CV Accuracy = 0.9558, Test Accuracy = 0.9649

Evaluating n_components=6...

n_components=6, LR: CV Accuracy = 0.9405, Test Accuracy = 0.9561
n_components=6, LDA: CV Accuracy = 0.9384, Test Accuracy = 0.9474
n_components=6, KNN: CV Accuracy = 0.9252, Test Accuracy = 0.9386
n_components=6, CART: CV Accuracy = 0.9228, Test Accuracy = 0.9474
n_components=6, NB: CV Accuracy = 0.9054, Test Accuracy = 0.9298
n_components=6, SVM: CV Accuracy = 0.9559, Test Accuracy = 0.9649

Evaluating n_components=7...

n_components=7, LR: CV Accuracy = 0.9427, Test Accuracy = 0.9649
n_components=7, LDA: CV Accuracy = 0.9449, Test Accu

 # PCA feature Selection With Random Search CV

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models and Hyperparameters for Grid Search
models = {
    'LR': (LogisticRegression(max_iter=1000), {'C': np.logspace(-4, 4, 10)}),
    'LDA': (LinearDiscriminantAnalysis(), {}),
    'KNN': (KNeighborsClassifier(), {'n_neighbors': range(1, 30)}),
    'CART': (DecisionTreeClassifier(), {'max_depth': range(1, 20)}),
    'NB': (GaussianNB(), {}),
    'SVM': (SVC(), {'C': np.logspace(-4, 4, 10), 'kernel': ['linear', 'rbf']})
}

cv_folds = 10
best_overall_score = 0
best_n_components = None
best_model_name = None

# ✅ Grid Search over the number of principal components
for n_components in range(5, 28):
    print(f"\nEvaluating n_components={n_components}...\n")
    
    pca = PCA(n_components=n_components)
    X_train_selected = pca.fit_transform(X_train)
    X_test_selected = pca.transform(X_test)

    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, (model, param_grid) in models.items():
        if param_grid:
            grid_search = GridSearchCV(model, param_grid=param_grid, cv=cv_folds, scoring='accuracy')
            grid_search.fit(X_train_selected, y_train)
            best_model = grid_search.best_estimator_
            mean_accuracy = grid_search.best_score_
        else:
            mean_accuracy = np.mean(cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy'))
            model.fit(X_train_selected, y_train)
            best_model = model

        y_pred = best_model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"n_components={n_components}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_n_components = n_components
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with n_components={best_n_components}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating n_components=5...

n_components=5, LR: CV Accuracy = 0.9428, Test Accuracy = 0.9649
n_components=5, LDA: CV Accuracy = 0.9427, Test Accuracy = 0.9474
n_components=5, KNN: CV Accuracy = 0.9384, Test Accuracy = 0.9386
n_components=5, CART: CV Accuracy = 0.9316, Test Accuracy = 0.9386
n_components=5, NB: CV Accuracy = 0.9054, Test Accuracy = 0.9386
n_components=5, SVM: CV Accuracy = 0.9603, Test Accuracy = 0.9561

Evaluating n_components=6...

n_components=6, LR: CV Accuracy = 0.9450, Test Accuracy = 0.9649
n_components=6, LDA: CV Accuracy = 0.9384, Test Accuracy = 0.9474
n_components=6, KNN: CV Accuracy = 0.9341, Test Accuracy = 0.9649
n_components=6, CART: CV Accuracy = 0.9294, Test Accuracy = 0.9737
n_components=6, NB: CV Accuracy = 0.9054, Test Accuracy = 0.9298
n_components=6, SVM: CV Accuracy = 0.9537, Test Accuracy = 0.9649

Evaluating n_components=7...

n_components=7, LR: CV Accuracy = 0.9472, Test Accuracy = 0.9649
n_components=7, LDA: CV Accuracy = 0.9449, Test Accu

# RFE Feature Selection with Grid Search CV

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models
models = [
    ('LR', LogisticRegression(max_iter=1000)),
    ('LDA', LinearDiscriminantAnalysis()),
    ('KNN', KNeighborsClassifier()),
    ('CART', DecisionTreeClassifier()),
    ('NB', GaussianNB()),
    ('SVM', SVC())
]

cv_folds = 10  # Use 10-fold cross-validation
best_overall_score = 0
best_n_features = None
best_model_name = None

# ✅ Iterate over the number of features to test using RFE
for n_features in range(5, 28):
    print(f"\nEvaluating n_features={n_features}...\n")
    
    # Feature Selection using RFE
    estimator = LogisticRegression(max_iter=1000)  # Use Logistic Regression as estimator for RFE
    rfe = RFE(estimator, n_features_to_select=n_features)
    X_train_selected = rfe.fit_transform(X_train, y_train)
    X_test_selected = rfe.transform(X_test)

    # Standardize After Feature Selection
    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, model in models:
        # Perform Cross-Validation
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy')
        mean_accuracy = np.mean(cv_scores)

        # Train on full training set
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"n_features={n_features}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        # ✅ Track best-performing model
        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_n_features = n_features
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with n_features={best_n_features}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating n_features=5...

n_features=5, LR: CV Accuracy = 0.9428, Test Accuracy = 0.9561
n_features=5, LDA: CV Accuracy = 0.9231, Test Accuracy = 0.9386
n_features=5, KNN: CV Accuracy = 0.9295, Test Accuracy = 0.9561
n_features=5, CART: CV Accuracy = 0.9208, Test Accuracy = 0.9474
n_features=5, NB: CV Accuracy = 0.9274, Test Accuracy = 0.9561
n_features=5, SVM: CV Accuracy = 0.9428, Test Accuracy = 0.9649

Evaluating n_features=6...

n_features=6, LR: CV Accuracy = 0.9472, Test Accuracy = 0.9649
n_features=6, LDA: CV Accuracy = 0.9406, Test Accuracy = 0.9386
n_features=6, KNN: CV Accuracy = 0.9405, Test Accuracy = 0.9386
n_features=6, CART: CV Accuracy = 0.9298, Test Accuracy = 0.9035
n_features=6, NB: CV Accuracy = 0.9318, Test Accuracy = 0.9474
n_features=6, SVM: CV Accuracy = 0.9318, Test Accuracy = 0.9561

Evaluating n_features=7...

n_features=7, LR: CV Accuracy = 0.9472, Test Accuracy = 0.9649
n_features=7, LDA: CV Accuracy = 0.9384, Test Accuracy = 0.9386
n_features=7, KNN: C

# RFE feature Selection With Rndom Search CV

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform

# ✅ Load Dataset
X = df.iloc[:, 2:36]  # Features
Y = df.iloc[:, 1]  # Target Variable

# ✅ Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Define Models and Hyperparameters for Random Search
models = {
    'LR': (LogisticRegression(max_iter=1000), {'C': uniform(0.0001, 1000)}),
    'LDA': (LinearDiscriminantAnalysis(), {}),
    'KNN': (KNeighborsClassifier(), {'n_neighbors': randint(1, 30)}),
    'CART': (DecisionTreeClassifier(), {'max_depth': randint(1, 20)}),
    'NB': (GaussianNB(), {}),
    'SVM': (SVC(), {'C': uniform(0.0001, 1000), 'kernel': ['linear', 'rbf']})
}

cv_folds = 10
n_iter_search = 10  # Number of random samples
best_overall_score = 0
best_n_features = None
best_model_name = None

# ✅ Random Search over the number of features using RFE
for n_features in np.random.randint(5, 28, 5):  # Randomly selecting 5 values of n_features
    print(f"\nEvaluating n_features={n_features}...\n")
    
    estimator = LogisticRegression(max_iter=1000)
    rfe = RFE(estimator, n_features_to_select=n_features)
    X_train_selected = rfe.fit_transform(X_train, y_train)
    X_test_selected = rfe.transform(X_test)

    sc = StandardScaler()
    X_train_selected = sc.fit_transform(X_train_selected)
    X_test_selected = sc.transform(X_test_selected)

    for name, (model, param_dist) in models.items():
        if param_dist:
            random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=n_iter_search, cv=cv_folds, scoring='accuracy', random_state=42)
            random_search.fit(X_train_selected, y_train)
            best_model = random_search.best_estimator_
            mean_accuracy = random_search.best_score_
        else:
            mean_accuracy = np.mean(cross_val_score(model, X_train_selected, y_train, cv=cv_folds, scoring='accuracy'))
            model.fit(X_train_selected, y_train)
            best_model = model

        y_pred = best_model.predict(X_test_selected)
        test_accuracy = accuracy_score(y_test, y_pred)

        print(f"n_features={n_features}, {name}: CV Accuracy = {mean_accuracy:.4f}, Test Accuracy = {test_accuracy:.4f}")

        if mean_accuracy > best_overall_score:
            best_overall_score = mean_accuracy
            best_n_features = n_features
            best_model_name = name

# ✅ Final Results
print(f"\n🎯 Best Model: {best_model_name} with n_features={best_n_features}")
print(f"📈 Best Cross-Validation Accuracy: {best_overall_score:.4f}")



Evaluating n_features=7...

n_features=7, LR: CV Accuracy = 0.9495, Test Accuracy = 0.9649
n_features=7, LDA: CV Accuracy = 0.9384, Test Accuracy = 0.9386
n_features=7, KNN: CV Accuracy = 0.9429, Test Accuracy = 0.9474
n_features=7, CART: CV Accuracy = 0.9209, Test Accuracy = 0.9211
n_features=7, NB: CV Accuracy = 0.9318, Test Accuracy = 0.9474
n_features=7, SVM: CV Accuracy = 0.9604, Test Accuracy = 0.9737

Evaluating n_features=9...

n_features=9, LR: CV Accuracy = 0.9604, Test Accuracy = 0.9737
n_features=9, LDA: CV Accuracy = 0.9516, Test Accuracy = 0.9561
n_features=9, KNN: CV Accuracy = 0.9473, Test Accuracy = 0.9561
n_features=9, CART: CV Accuracy = 0.9385, Test Accuracy = 0.9298
n_features=9, NB: CV Accuracy = 0.9208, Test Accuracy = 0.9561
n_features=9, SVM: CV Accuracy = 0.9583, Test Accuracy = 0.9737

Evaluating n_features=22...

n_features=22, LR: CV Accuracy = 0.9583, Test Accuracy = 0.9649
n_features=22, LDA: CV Accuracy = 0.9472, Test Accuracy = 0.9561
n_features=22, KN

# FS Feature Selection For Grid Search CV